In [1]:
import pandas as pd
import torch
from torch.utils import data
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision 
from torchvision import transforms 
import torch.optim as optim
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,plot_confusion_matrix



In [2]:
def getData(mode):
    if mode == 'train':
        img = pd.read_csv('train_img.csv')
        label = pd.read_csv('train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('test_img.csv')
        label = pd.read_csv('test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)

In [3]:
class RetinopathyLoader(data.Dataset):
    def __init__(self, root, mode):
    
        self.root = root
        # the type is numpy.ndarray
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, index):
        #get the path and load the img 
        path = self.root + self.img_name[index] + '.jpeg' 
#         img  = mpimg.imread(path)
        img  = Image.open(path).convert('RGB')

        #get label according to indx
        label = self.label[index]
        
        #set the trasnform
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5)
        ])
        
        # Use transform make 
        img = transform(img)
        # make the size 3*512*512 -> 1*3*512*512
#         img = torch.unsqueeze(img, 0)
        # get the tensor form label
        label = torch.from_numpy(np.array(label))

#         img = transforms.Normalize(img,(0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#         img = torchvision.transforms.ToPILImage(img)
#         img = torchvision.transforms.ToTensor()(img)

        return img, label

train_data = RetinopathyLoader("data/", "train")
test_data  = RetinopathyLoader("data/", "test")

train_data = DataLoader(train_data,batch_size=4,num_workers = 16,pin_memory = True)
test_data = DataLoader(test_data,batch_size=4,num_workers = 16,pin_memory = True)

# print(train_data[0][0].shape)
# transform = transforms.Compose([transforms.ToPILImage()])
# transforms.ToPILImage()(train_data[0][0][0]).show()
# transform(train_data[0][0][0]).show()

> Found 28099 images...
> Found 7025 images...


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
class ResNet(nn.Module):
    def __init__(self, m_type,pretrained=True):
        super(ResNet, self).__init__()

        self.classify = nn.Linear(2048, 5)

        pretrained_model = torchvision.models.__dict__['resnet{}'.format(m_type)](pretrained=pretrained)
        self.conv1 = pretrained_model._modules['conv1']
        self.bn1 = pretrained_model._modules['bn1']
        self.relu = pretrained_model._modules['relu']
        self.maxpool = pretrained_model._modules['maxpool']

        self.layer1 = pretrained_model._modules['layer1']
        self.layer2 = pretrained_model._modules['layer2']
        self.layer3 = pretrained_model._modules['layer3']
        self.layer4 = pretrained_model._modules['layer4']

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Linear(
            pretrained_model._modules['fc'].in_features, 5
        )
        
        del pretrained_model

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
#         print("before:",x.shape)
        x = x.view(x.size(0), -1)
#         print("after:",x.shape)
        x = self.fc(x)
#         print("final:",x.shape)


        return x

In [6]:
def ResNet18_train(net):
    true_ans = 0.0
    false_ans = 0.0
    confusion_yTrue = []
    confusion_yPred = []
    for idx,pair in enumerate(train_data):
        net.train()
        inputs,labels = pair
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)

        # make gradients be zero , default is accumulated 
        optimizer.zero_grad()
        #put the input to net , output size 1*1000 
        outputs = net.forward(inputs)

        #caclulate lose
        loss = criterion(outputs, labels)
#         print(loss.item())
        #caculate gradient
        loss.backward()
        #using gradient update weight
        optimizer.step()
        #weight decay
        for i in range(len(labels)):
            ground_true = labels[i].item()
            pred_y = torch.argmax(outputs[i]).item()
    #         print("pred= ",pred_y)
            confusion_yTrue.append(ground_true)
            confusion_yPred.append(pred_y)
        
            if ground_true == pred_y:
                true_ans = true_ans + 1
            else:
                false_ans = false_ans + 1
                
        if(idx!=0 and idx%100==0):
            print("idx = ",idx,"Train_accuracy = ",true_ans/(false_ans+true_ans))
    return true_ans, false_ans, confusion_yTrue, confusion_yPred



In [7]:
def evalutation(net):
    net.eval()
    true_ans = 0.0
    false_ans = 0.0
    confusion_yTrue = []
    confusion_yPred = []
    for idx,pair in enumerate(test_data):
        inputs,labels = pair
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)

        outputs = net(inputs)

        for i in range(len(labels)):
            ground_true = labels[i].item()
            pred_y = torch.argmax(outputs[i]).item()
    #         print("pred= ",pred_y)
            confusion_yTrue.append(ground_true)
            confusion_yPred.append(pred_y)
            if ground_true == pred_y:
                true_ans = true_ans + 1
            else:
                false_ans = false_ans + 1
                
        if(idx!=0 and idx%100==0):
            print("idx = ",idx,"Eval_accuracy = ",true_ans/(false_ans+true_ans))
            
    return true_ans, false_ans, confusion_yTrue, confusion_yPred

In [8]:
def validation(net,path):
    print("Test the :",path)
    net.load_state_dict(torch.load(path))
    net.eval()
    true_ans = 0.0
    false_ans = 0.0
    confusion_yTrue = []
    confusion_yPred = []
    for idx,pair in enumerate(test_data):
        inputs,labels = pair
        inputs = inputs.float().to(device)
        labels = labels.long().to(device)

        outputs = net(inputs)

        for i in range(len(labels)):
            ground_true = labels[i].item()
            pred_y = torch.argmax(outputs[i]).item()
    #         print("pred= ",pred_y)
            confusion_yTrue.append(ground_true)
            confusion_yPred.append(pred_y)
            if ground_true == pred_y:
                true_ans = true_ans + 1
            else:
                false_ans = false_ans + 1
    if(idx!=0 and idx%100==0):
        print("idx = ",idx,"Eval_accuracy = ",true_ans/(false_ans+true_ans))
    return true_ans/(false_ans+true_ans)


In [9]:
# Training
flag =""
def run(net):
    global flag
    if(net==pre_net):
        print("pretrain network:")
        flag ="pre"
    else:
        print("newtrain network:")
        flag ="new"
    train_epoch_list = []
    train_acc_list = []
    test_epoch_list = []
    test_acc_list = []
    f= open('ResNet18.txt','a')
    for epoch in range(20):
        print ("Train: epoch "+ str(epoch+1))
        start = time.time()

        #Traning
        net.train()
        train_epoch_list.append(epoch)
#         result = ResNet18_train(net,len(train_data))
        result = ResNet18_train(net)
        train_y_true = result[2]
        train_y_pred = result[3]
        ACC = (result[0]/(result[0]+result[1]))
        train_acc_list.append(ACC)
        f.write(str(ACC)+" ")
        print ("TrainAccuracy is : "+str(ACC))

        
        #Testing
        net.eval()
        test_epoch_list.append(epoch)
#         result = evalutation(net,test_data,len(test_data))
        result = evalutation(net)
        test_y_true = result[2]
        test_y_pred = result[3]
        ACC = (result[0]/(result[0]+result[1]))
        test_acc_list.append(ACC)
        f.write(str(ACC)+"\n")
        print ("TestAccuracy is : "+str(ACC))
        end = time.time()
        print("執行時間：%f 秒" % (end - start))
        if(ACC==max(test_acc_list)):
            torch.save(net.state_dict(), "CKPT/"+flag+"_ResNet18_final_ckpt")
    f.close()
    return train_y_true,train_y_pred,test_y_true,test_y_pred,train_acc_list,test_acc_list

In [10]:
import numpy as np
def plot_confusion_matrix(y_true,y_pred,title):
    print(title+":")
    cm = confusion_matrix(y_true,y_true,labels=[0,1,2,3,4])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1,2,3,4])
    disp.plot(cmap=plt.cm.Blues)

In [11]:
def plotImg():
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")

    plt.plot(new_tran_accu,color="blue",label="new_tran_accu")
    plt.plot(new_test_accu,color="orange",label="new_test_accu")
    plt.plot(pretrain_tran_accu,color="green",label="pretrain_tran_accu")
    plt.plot(pretrain_test_accu,color="red",label="pretrain_test_accu")
    plt.title("ResNet18")
    plt.legend()


In [12]:
load=0
pre_net = ResNet(18,True).to(device)
net = ResNet(18,False).to(device)

if(load==0):
    #pretrain net
    criterion = F.cross_entropy
    optimizer = torch.optim.SGD(pre_net.parameters(), lr=1e-3,momentum=0.1,weight_decay=5e-4)
    result_old = run(pre_net)
    confusion_matrix(result_old[0],result_old[1],labels=[0,1,2,3,4])
    plot_confusion_matrix(result_old[0],result_old[0],title="pretrain_train_confusion")
    plot_confusion_matrix(result_old[2],result_old[3],title="pretrain_test_confusion")
    pretrain_tran_accu = result_old[4]
    pretrain_test_accu = result_old[5]

    #new net
    criterion = F.cross_entropy
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-3,momentum=0.1,weight_decay=5e-4)
    result_new = run(net)
    plot_confusion_matrix(result_new[0],result_new[1],title="new_train_confusion")
    plot_confusion_matrix(result_new[2],result_new[3],title="new_test_confusion")
    new_tran_accu = result_new[4]
    new_test_accu = result_new[5]

else:
    path = "CKPT/pre_ResNet18_final_ckpt"
    print(validation(pre_net,path))
    path = "CKPT/new_ResNet18_final_ckpt"
    print(validation(net,path))


pretrain network:
Train: epoch 1
idx =  100 Train_accuracy =  0.7698019801980198
idx =  200 Train_accuracy =  0.7524875621890548
idx =  300 Train_accuracy =  0.7416943521594684
idx =  400 Train_accuracy =  0.7412718204488778
idx =  500 Train_accuracy =  0.7410179640718563
idx =  600 Train_accuracy =  0.7400166389351082
idx =  700 Train_accuracy =  0.7382310984308131
idx =  800 Train_accuracy =  0.7393882646691635
idx =  900 Train_accuracy =  0.7413984461709212
idx =  1000 Train_accuracy =  0.743006993006993
idx =  1100 Train_accuracy =  0.7411444141689373
idx =  1200 Train_accuracy =  0.7404246461282264
idx =  1300 Train_accuracy =  0.7390468870099923
idx =  1400 Train_accuracy =  0.7380442541042113
idx =  1500 Train_accuracy =  0.7348434377081945
idx =  1600 Train_accuracy =  0.7343847595252967
idx =  1700 Train_accuracy =  0.7338330393885949
idx =  1800 Train_accuracy =  0.7345918933925597
idx =  1900 Train_accuracy =  0.7354024197790636
idx =  2000 Train_accuracy =  0.73488255872063

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "/home/frank/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/frank/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/frank/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-3-223d94ca7e15>", line 17, in __getitem__
    img  = Image.open(path).convert('RGB')
  File "/home/frank/miniconda3/envs/pytorch/lib/python3.9/site-packages/PIL/Image.py", line 2912, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'data/36644_left.jpe

In [ ]:
# need to set load = 0
plotImg()
